# Coursera Capstone - FINAL PROJECT - Where to open a pizzeria in Zürich (Switzerland)
Author: Nicolo' (Nick) Sgobba

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('All libraries imported successfully.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
All libraries imported successfully.


In [2]:
# Data Source
wikipediaPage = requests.get("https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich")

In [3]:
# import it in a soup object
soup = BeautifulSoup(wikipediaPage.content, 'html5lib')

# get the table with the needed info
table = soup.find_all('table')[1]

# extract all <tr> from the table
trs = table.find_all('tr')

In [4]:
# function to parse the <tr> and return an entry for the dataframe : return Borough, Neighborhood
def parseTableRow(tr, index):
    borough = 'District ' + str(index)
    
    neighborhood = ''

    td = tr.find_next('td')
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    
    for string in td.stripped_strings:
        if (neighborhood != ''):
            neighborhood = neighborhood + ', '
        neighborhood = neighborhood + string
    
    return borough, neighborhood

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood


In [6]:
# fill the dataframe
index = 1
for tr in trs:
    if 'District ' in str(tr):
        data = parseTableRow(tr, index)
        neighborhoods = neighborhoods.append({'Borough': data[0], 'Neighborhood': data[1]}, ignore_index=True)
        index = index + 1

In [7]:
neighborhoods

,Borough,Neighborhood
0,District 1,"Rathaus, Hochschulen, Lindenhof, City"
1,District 2,"Wollishofen, Leimbach, Enge"
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld"
3,District 4,"Werd, Langstrasse, Hard"
4,District 5,"Gewerbeschule, Escher Wyss"
5,District 6,"Unterstrass, Oberstrass"
6,District 7,"Fluntern, Hottingen, Hirslanden, Witikon"
7,District 8,"Seefeld, Mühlebach, Weinegg"
8,District 9,"Albisrieden, Altstetten"
9,District 10,"Höngg, Wipkingen"


In [48]:
CLIENT_ID = 'OHOJGBQX3OYUYLZLB53Z2DIRCUPNMYDD4CXKUJNWWMP12H15'
CLIENT_SECRET = 'TPO3OZUZX2ABG5XUJ4P5LCTCYWKWZGEVVNS1O1PGLVE305OM'
VERSION = '20180604'

#print('Credentials:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

baseUrl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION
)

In [49]:
def getLocation(district):

    url = (baseUrl+'&address={}').format('Zürich'+district)
    response = requests.get(url).json() # get response
    
    success = False
    retries = 5
    while (not success) and retries > 0:
        try:
            geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
            success = True
        except: 
            retries = retries - 1
    
    if (retries == 0):
        print('ERROR while getting data. Please retry!')
        return
    
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    return latitude, longitude

In [59]:
nh = neighborhoods.copy()

latitudeCln = []
longitudeCln = []

for index, row in nh.iterrows():
    print(row[0])
    lat, long = getLocation(row[0])
    latitudeCln.append(lat)
    longitudeCln.append(long)

nh['Latitude'] = latitudeCln
nh['Longitude'] = longitudeCln

nh.shape

District 1
ERROR while getting data. Please retry!


TypeError: 'NoneType' object is not iterable

In [25]:
nh

,Borough,Neighborhood
0,District 1,"Rathaus, Hochschulen, Lindenhof, City"
1,District 2,"Wollishofen, Leimbach, Enge"
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld"
3,District 4,"Werd, Langstrasse, Hard"
4,District 5,"Gewerbeschule, Escher Wyss"
5,District 6,"Unterstrass, Oberstrass"
6,District 7,"Fluntern, Hottingen, Hirslanden, Witikon"
7,District 8,"Seefeld, Mühlebach, Weinegg"
8,District 9,"Albisrieden, Altstetten"
9,District 10,"Höngg, Wipkingen"


In [26]:
address = 'Zürich, Switzerland'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zürich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zürich are 47.3723957, 8.5423216.


In [27]:
# make a copy of the dataframe to get it simply back if needed
nhz = nh.copy()

In [28]:
# create map of Zürich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(nhz['Latitude'], nhz['Longitude'], nhz['Borough'], nhz['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

KeyError: 'Latitude'

In [29]:
# function to repeat the exploring process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [30]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

In [31]:
# Use category id 4bf58dd8d48988d1ca941735 to only get the pizzerie
zh_venues_pizzeria = getNearbyVenues(names=nhz['Neighborhood'], latitudes=nhz['Latitude'], longitudes=nhz['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ca941735')
zh_venues_pizzeria.head()

KeyError: 'Latitude'

In [32]:
# function to add markers for given venues to ma
def addToMap(df, color, existingMap):
    for lat, lng, neighborhood, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            parse_html=False).add_to(existingMap)

In [33]:
map_zh_pizzerie = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(zh_venues_pizzeria, 'red', map_zh_pizzerie)
map_zh_pizzerie

NameError: name 'zh_venues_pizzeria' is not defined

In [34]:
zh_venues_schools = getNearbyVenues(names=nhz['Neighborhood'], latitudes=nhz['Latitude'], longitudes=nhz['Longitude'], radius=1000, categoryIds='4d4b7105d754a06372d81259')
zh_venues_schools.head()

KeyError: 'Latitude'

In [35]:
map_zh_schools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(zh_venues_schools, 'green', map_zh_schools)
map_zh_schools

NameError: name 'zh_venues_schools' is not defined

In [36]:
zh_venues_enterprises = getNearbyVenues(names=nhz['Neighborhood'], latitudes=nhz['Latitude'], longitudes=nhz['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
zh_venues_enterprises.head()

KeyError: 'Latitude'

In [37]:
map_zh_enterprises = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(zh_venues_enterprises, 'yellow', map_zh_enterprises)
map_zh_enterprises

NameError: name 'zh_venues_enterprises' is not defined

In [38]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [39]:
zh_data = nhz.copy()
addColumn(zh_data, 'Pizzerie', zh_venues_pizzeria)
addColumn(zh_data, 'Schools', zh_venues_schools)
addColumn(zh_data, 'Enterprises', zh_venues_enterprises)
zh_data

NameError: name 'zh_venues_pizzeria' is not defined

In [40]:
# negative weight, because Paolo wants to open a pizzeria and thus wants to avoid concurrence as much as possible
weight_pizzerie = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight because employees are even better customers
weight_enterprises = 1.5

In [41]:
zh_weighted = zh_data[['Borough', 'Neighborhood']].copy()

In [42]:
zh_weighted['Score'] = zh_data['Pizzerie'] * weight_pizzerie + zh_data['Schools'] * weight_schools + zh_data['Enterprises'] * weight_enterprises
zh_weighted = zh_weighted.sort_values(by=['Score'], ascending=False)
zh_weighted

KeyError: 'Pizzerie'

In [43]:
map_zh_result = folium.Map(location=[latitude, longitude], zoom_start=12)

nhz_win = nhz[nhz['Borough'] == 'District 10']

for lat, lng, borough, neighborhood in zip(nhz_win['Latitude'], nhz_win['Longitude'], nhz_win['Borough'], nhz_win['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh_result) 

addToMap(zh_venues_pizzeria[zh_venues_pizzeria['Neighborhood'] == 'Höngg, Wipkingen'], 'red', map_zh_result)
addToMap(zh_venues_schools[zh_venues_schools['Neighborhood'] == 'Höngg, Wipkingen'], 'green', map_zh_result)
addToMap(zh_venues_enterprises[zh_venues_enterprises['Neighborhood'] == 'Höngg, Wipkingen'], 'yellow', map_zh_result)

map_zh_result

KeyError: 'Latitude'